In [ ]:
#!/usr/bin/env python
# coding: utf-8

from pymongo import MongoClient, GEOSPHERE
from os import getenv

In [ ]:
password = getenv('MONGODB_ROOT_PASSWORD')
client = MongoClient('mongodb://root:{}@my-mongodb:27017'.format(password))
db = client['geotest-db']

In [ ]:
# print(getenv('MONGODB_ROOT_PASSWORD'))

In [ ]:
print(db.list_collection_names())

collection = db['geotest-collection']
collection.create_index([('location', GEOSPHERE)])

print(collection.index_information())

In [ ]:
# for x in range(20000):
#     for y in range(20000):
#         collection.insert_one({
#             'name': "point %d - %d" % (x,y),
#             'location' : [x % 180, y % 90]
#         })


points = collection.find({'location': {'$nearSphere' : [3,7]}}).limit(5)

for p in points:
    print(p)

In [ ]:
points = collection.find({'location': {'$within': {'$box': [[2,2], [6,6]]}}})

total = 0
for p in points:
    print(p)
    total += 1
print(total)

In [ ]:
from kafka import KafkaConsumer
from json import loads
from pymongo import MongoClient, GEOSPHERE
from os import getenv

In [ ]:
password = getenv('MONGODB_ROOT_PASSWORD')
client = MongoClient('mongodb://root:{}@my-mongodb:27017'.format(password))
db = client['geotest-db']

In [ ]:
consumer = KafkaConsumer(
  '2020_pop_geojson_covid19',
  bootstrap_servers=['my-kafka:9092'], 
  auto_offset_reset='earliest',
  enable_auto_commit=False,
  value_deserializer= lambda x: loads(x.decode('utf-8'))
)

In [ ]:
geo_collection = db['2020_pop_geojson']

In [ ]:
geo_collection.create_index([('geometry', GEOSPHERE)])

In [ ]:
break_point = 10
counter = 0
for message in consumer:
  body = message.value
  print("received a new message: ", body)
  geo_collection.insert(body)
    
#   collection.insert({"date_received": date.today().__str__(), "data": body})

In [ ]:
consumer.subscribe('2020_pop_geojson')

In [ ]:
features = geo_collection.find({'geometry' : {'$within': {'$box' : [[100, 100],[0,0]]}}}).limit(5)

for f in features:
    print(f)

In [ ]:
features = geo_collection.find({'geometry' : {'$nearSphere': [62.21541, 34.34125]}}).limit(5)

for f in features:
    print(f)